# Visualización II: trabajando con mapas

En este TP vamos a trabajar primordialmente con mapas y datos geográficos. Para esto se utilizará una libreria relacionada a pandas, llamada geopandas. Primero instalamos todas las librerias necesarias.

In [ ]:
!pip3 install qeds fiona geopandas xgboost gensim folium pyLDAvis descartes

Esto puede ser un poco más pesado que en los TPs anteriores. Ahora cargamos las librerias ya descargadas.

In [ ]:
!pip3 install geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

from shapely.geometry import Point

%matplotlib inline
import qeds


## Formateando los datos
En este caso, vamos a utilizar un nuevo paquete llamado geopandas para crear mapas.

Los mapas son bastante complicados... 

Afortunadamente, geopandas es un herramienta muy versátil e intuitiva.

Comencemos con un DataFrame que tenga las coordenadas de latitud y longitud de varias ciudades de Sudamérica.

Nuestro objetivo es convertirlas en algo que podamos graficar - en este caso, un GeoDataFrame.


In [ ]:
df = pd.DataFrame({
    'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
    'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
    'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
    'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]
})

Para mapear las grandes ciudades debemos unir latitud y longitud en lo que llamaremos "coordenadas".


In [ ]:
df["Coordinates"] = list(zip(df.Longitude, df.Latitude))
df.head()

El proximo paso es convertir esas coordenadas en un formato que pueda utilizarse para geolocalizar. Este se llama `Shapely` `Point` object.

Esto se puede hacer aplicando el método `Point` a las `Coordinates`.

In [ ]:
df["Coordinates"] = df["Coordinates"].apply(Point)
df.head()

Finalmente el data.frame se convierte a lo que se conoce como un geodataframe llamando a la función de geopandas. Es necesario especificarle en que columna se encuentran las coordenas. 

Geodataframe es muy parecido a un dataframe común, como cualquiera de los que vinimos trabajando, pero contiene información que permite llevar a cabo mapas.



In [ ]:
gdf = gpd.GeoDataFrame(df, geometry="Coordinates")
gdf.head()

## Graficando el mapa

Ahora con GeoDataFrame ya podemos avanzar.

Esto implicará tres pasos

1. Obtener el mapa  
2. Graficar el mapa
3. Graficar puntos o áreas sobre el mapa

Iremos por cada uno de estos pasos a continuación

### 1. Get the map

Una organización llamada [Natural Earth](https://www.naturalearthdata.com/)  compiló los datos del mapa que utilizamos acá.

El archivo proporciona los contornos de los países, sobre los cuales graficaremos las ubicaciones de las ciudades de nuestro GeoDataFrame.

Geopandas ya viene con estos datos incluidos, así que no tenemos que buscarlos





In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world = world.set_index("iso_a3")

world.head()

`world` es un geodataframe con las siguientes columnas:

- `pop_est`: contiene la población estimada del país.
- `continent`: el continente del país
- `name`: el nombre del país
- `iso_a3`: las tres letras que definen al pais  
- `gdp_md_est`: una estimación del GDP del país
- `geometry`: un `POLYGON` de cada país, ya se analizará esto en detalle más adelante.

In [ ]:
world.geometry.name

Observa que la geometría de este GeoDataFrame se almacena en la columna geometry.

Una breve nota acerca de los polígonos

En lugar de puntos (como lo son nuestras ciudades), los objetos de geometría ahora son polígonos.

Un polígono es lo que probablemente ya pienses que es: una colección de puntos ordenados conectados por líneas rectas.

Cuanto menor sea la distancia entre los puntos, más fácilmente el polígono puede aproximar formas no lineales.

Veamos un ejemplo de un polígono.


In [ ]:
world.loc["ALB", 'geometry']

Este país es Albania

In [ ]:
x, y = world.loc["ALB", "geometry"].exterior.coords.xy

print('Points in the exterior of Albania:', len(x))

El siguiente ¿cuál es?

In [ ]:
world.loc["AFG", "geometry"]

In [ ]:
x, y = world.loc["AFG", 'geometry'].exterior.coords.xy

print('Points in the exterior of Afghanistan:', len(x))

Notese que Afganistán es más complejo que Albania, por lo tanto necesita más puntos.

### 2. Graficando el mapa

In [ ]:
fig, gax = plt.subplots(figsize=(10,10))

world.query("continent == 'South America'").plot(ax=gax, edgecolor='black',color='white')

gax.set_xlabel('longitud')
gax.set_ylabel('latitud')

gax.spines['top'].set_visible(False)
gax.spines['right'].set_visible(False)

plt.show()

Hay mucho trabajo detrás del mapa que acabamos de hacer, desde definiciones provenientes de las Ciencias Geográficas hasta la programación.

### 3. Graficar los puntos o áreas en el mapa

En el código a continuación, ejecutamos los mismos comandos que antes para graficar los países de Sudamérica, pero ahora también graficamos los datos en gdf, que contiene la ubicación de las ciudades de Sudamérica.

In [ ]:
fig, gax = plt.subplots(figsize=(10,10))

world.query("continent == 'South America'").plot(ax = gax, edgecolor='black', color='white')

gdf.plot(ax=gax, color='red', alpha = 0.5)

gax.set_xlabel('longitud')
gax.set_ylabel('latitud')
gax.set_title('Sud America')

gax.spines['top'].set_visible(False)
gax.spines['right'].set_visible(False)

plt.show()

Adhiriendo leyendas a los puntos.
Es necesario para saber que ciudad es cada una

In [ ]:
fig, gax = plt.subplots(figsize=(10,10))

world.query("continent == 'South America'").plot(ax = gax, edgecolor='black', color='white')

gdf.plot(ax=gax, color='red', alpha = 0.5)

gax.set_xlabel('longitud')
gax.set_ylabel('latitud')
gax.set_title('Sud America')

gax.spines['top'].set_visible(False)
gax.spines['right'].set_visible(False)

for x, y, label in zip(gdf['Coordinates'].x, gdf['Coordinates'].y, gdf['City']):
    gax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points')

plt.show()

## Caso de estudio: analizando la Ciudad de Buenos Aires 

Vamos a analizar un ejemplo puntual, el de la Ciudad de Buenos Aires. No analizamos PBA dado que probablemente sea utilizado en sus TPs ;) pero el procedimiento es muy similar

In [ ]:
!apt install libspatialindex-dev
!pip install rtree
import pandas as pd
import geopandas
import shapely.wkt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

La información geográfica generalmente se agrupa en dos grandes categorías:

- Datos ráster
- Datos vectoriales

**Ráster**: Los datos ráster son aquellos donde la información se encuentra codificada sobre una grilla o red, donde cada celda (o píxel) presenta un valor numérico. Los datos ráster son usados comúnmente en imágenes satelitales, en este caso cada píxel representa un espacio definido por la granularidad o calidad de la imagen tomada, por ejemplo, un píxel puede representar una superficie de 30 metros por 30 metros.

Sin embargo, en esta clase nos vamos a concentrar en los datos **vectoriales**. En los datos vectoriales contamos con los siguientes tipos de objetos básicos: puntos, líneas, polígonos; así también tenemos multipunto, multilínea y multipolígono. La información se estructura a partir de las coordenadas de los vértices de cada uno de esos objetos.

La datos vectoriales comúnmente se presenta en archivos como el formato shapefile (.shp) de ESRI, archivos GeoJSON, bases de PostGIS (es una extensión del motor de bases de datos PostgreSQL), etc.
Con la librería **GeoPandas** se pueden leer muchos de estos archivos, interfaceando con la librería GDAL/OGR, usando la función geopandas.read_file.
Además, también es posible leer información geográfica desde archivos planos (.csv) donde tenemos las coordenadas de puntos que referencian algún objeto. Por último, también existen casos en los que la información está guardada en formato WKT (Well-Known Text).

En esta notebook vamos a ver distintas formas de importar información geográfica de archivos vectoriales.

In [ ]:
# https://data.buenosaires.gob.ar/dataset/barrios
barrios = pd.read_csv("/content/drive/MyDrive/IMD/TP5/barrios.csv", encoding='latin1', delimiter=';')

In [ ]:
barrios.head()

In [ ]:
# En este caso la información está en WKT
barrios["WKT"].iloc[0]

Los GeoDataFrame son objetos que tienen los mismos atributos que un Pandas DataFrame, con lo cual se puede manipular de datos a la que estamos acostumbrados. Como, por ejemplo, el método .plot(), acceso por .loc, .iloc, etc.
Además, los GeoDataFrames tienen un atributo *geometry* que indica la columna que contiene la información geográfica.

In [ ]:
def from_wkt(df, wkt_column, crs='EPSG:4326'):
    
    df["coordinates"]= df[wkt_column].apply(shapely.wkt.loads) # empleamos una función de shapely para leer WKT
        
    gdf = geopandas.GeoDataFrame(df, geometry='coordinates', crs=crs) # seteamos la columna de geometría
    
    return gdf

In [ ]:
barrios = from_wkt(barrios, "WKT")

Asi podemos ver info específica de los barrios

In [ ]:
barrios.query("BARRIO == 'CABALLITO'") # podemos usar query

También sus límites

In [ ]:
barrios.plot(); # BOOM: con uds. CABA!

Y también los polígonos

In [ ]:
barrios.geometry.head() 

La librería shapely nos provee acceso a los objetos geométricos. Con ella podemos generar los objetos Point, Polygon y Line. Veamos un ejemplo. Además, podemos especificar la proyección pasando el parámetro crs. Para ver cómo transformar una proyección se puede ir a: http://geopandas.readthedocs.io/en/latest/projections.html

In [ ]:
from shapely.geometry import Point

p = Point(-58.396295, -34.591789) # Armo un punto a partir de dos coordenadas geográficas

df = pd.DataFrame(data = {'id': [1]}) # Armo un DataFrame

gdf = geopandas.GeoDataFrame(df, crs="EPSG:4326", geometry=[p])

gdf # este es un GeoDataFrame con un solo punto

In [ ]:
fig, ax = plt.subplots() # Noten que vamos a reutilizar el axis (ax) en cada plot. 
ax.set_aspect('equal')
barrios.plot(ax=ax, color='white', edgecolor='black') # Esto es para evitar que las capas se ploteen separadas.
gdf.plot(ax=ax, marker='x', color='red', markersize=25)
plt.show();

Geopandas nos permite también realizar lo que se conoce como "operaciones geográficas". Básicamente es obtener datos, calcular áreas, distancias, entre otras. 

Las principales operaciones geográficas son:

    - equals
    - contains
    - crosses
    - disjoint
    - overlaps
    - touches
    - within
    - covers
    - contains
    - intersects
    - intersection
    - union
    - unary_union
    - difference
    - overlay (difference, intersection, symmetric_difference) 
    - sjoin
    
    
En el siguiente link se tiene una referencia de varios de estos métodos, los heredados de shapely:
https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships

Ahora vamos a mapear hospitales. Descargamos primero la base de datos:

In [ ]:
bomberos = pd.read_csv("/content/drive/MyDrive/IMD/TP5/bomberos.csv")
bomberos.head(3)

In [ ]:
bomberos_gdf = gpd.GeoDataFrame(bomberos, geometry=gpd.points_from_xy(bomberos['long'], bomberos['lat']))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
barrios.plot(ax=ax, color='white', edgecolor='black')
bomberos.plot(ax=ax, marker='o', color='red', markersize=25)
plt.show();

Imagínese que se desea bacer el siguiente análisis: suponiendo que cada delegación de bomberos tiene un alcance de hasta 1.5 km (ACLARACIÓN: este valor no se basa en la realidad) se desean obtener los lugares abarcados y los no abarcados por el servicio.

Para ello primero tendremos que convertir la proyección a una proyección que nos permita trabajar en metros y no en grados. Para eso usaremos la proyección  Gauss-Krueger Buenos Aires. 

Convertimos nuestros datasets a GKBA

In [ ]:
barrios.set_crs("EPSG:4326", inplace=True)
bomberos_gdf.set_crs("EPSG:4326", inplace=True)
print("CRS asignado")

In [ ]:
# Transformar el CRS de 'barrios'
barrios_gkba = barrios.to_crs(crs="+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
print("Transformación de CRS para barrios completada.")

# Transformar el CRS de 'bomberos'
bomberos_gkba = bomberos_gdf.to_crs(crs="+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
print("Transformación de CRS para bomberos completada.")

¿Qué pasó ahora con la geometría?

In [ ]:
bomberos_gkba.head().geometry

In [ ]:
bomberos_gkba_buff = bomberos_gkba.copy()
bomberos_gkba_buff.geometry = bomberos_gkba.buffer(1500)

Ahora graficamos el área cubierta

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
barrios_gkba.plot(ax=ax, color='white', edgecolor='black')
bomberos_gkba_buff.plot(ax=ax, color='red')
plt.show();

Ahora vamos a ver cómo calcular el área cubierta y el área no cubierta. El área cubierta es la **intersección** entre la capa de barrios y la capa de bomberos (con su buffer). Por otro lado, el área no cubierta es la **diferencia** entre la capa de barrios y la capa de bomberos (con su buffer).

In [ ]:
interseccion = geopandas.overlay(bomberos_gkba_buff, barrios_gkba, how = "intersection")

In [ ]:
interseccion.plot();

## Mapas interactivos
En esta sección vamos a trabajar con mapas interactivos. Suelen ser buenos para armar aplicaciones y mostrar, a diferencia de los anteriores que son primordialmente analíticos.

Acá se requieren otros paquetes

In [ ]:
!pip3 install folium
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


In [ ]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
m_1


Varios argumentos pueden ayudar a customizar la apareiencia del mapa:

La ubicación establece el centro inicial del mapa. Utilizamos la latitud (42,32 ° N) y longitud (-71,0589 ° E) de la ciudad de X.
Tiles cambia el estilo del mapa; en este caso, elegimos el estilo OpenStreetMap. Si tienes curiosidad, puedes encontrar otras opciones listadas aquí.
Zoom_start establece el nivel inicial de zoom del mapa, donde valores más altos acercan más el zoom al mapa.
Tómese el tiempo ahora para explorar acercando y alejando el zoom, o arrastrando el mapa en diferentes direcciones.


Los datos que vamos a utilizar se encuentran en el siguiente link: 
[Base de crímenes](https://drive.google.com/file/d/1AZaqiHmIA1J3zzjOdva5cf4Ao8tviCmN/view?usp=sharing)

Ahora vamos a analizar nuevos datos, incorporandolos ya saben como:

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

crimes = pd.read_csv("/content/drive/MyDrive/IMD/TP5/crime.csv", encoding='latin-1')


# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()


El próximo paso consiste en graficar puntos:

In [ ]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]
# Create a map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)

# Display the map
m_2


Y también se pueden utilizar algoritmos de clusterización o agrupamiento por distancia (ya veremos en detalle que es esto en un TP específico:

In [ ]:
import math
# Create the map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

# Display the map
m_3


También es posible realizar mapas de calor, muy utilizados en epidemiología:

In [ ]:
# Create a base map
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_5)

# Display the map
m_5

# Ejercicio
A este punto ya analizaron muchas formas diferentes de trabajar con mapas.
La propuesta es que lleven estos a la Provincia de Buenos Aires. Si tienen que inventarse bases de datos con latitud y longitud de distintos eventos (por ejemplo casos de dengue) ya saben que chatGPT es una herramienta bastante creativa ;) 

In [ ]:
# en sus manos